In [2]:
import json
from database_utils import Session; session = Session();
from collections import Counter, defaultdict

### Wait Time Data
Every row is represents a judicial appointment. It contains:
* All the judge's data
* Information about the appointment (court type)
* Congressional Start Year of the nomination.
* Nominating President
* congressional makeup

In [3]:

session.rollback()
wait_data = session.execute(
"""
SELECT
    j.*,
    con.start_year AS congress_start_year,
    appt.start_year AS appointent_year,
    appt.nomination_date,
    appt.days_to_confirm,
    appt.court_name,
    con.president_party_senate_majority_perc,
    con.president_party_senate_majority,
    con.party_of_president,
    con.president
FROM appointment appt
JOIN congress con
    ON date_part('year', appt.nomination_date) >= con.start_year
    AND date_part('year', appt.nomination_date) < con.end_year
JOIN judge j
    ON appt.nid = j.nid
WHERE appt.days_to_confirm IS NOT NULL
"""
)
wait_data_export = []
for row in wait_data:
    row = dict(row)
    row['nomination_date'] = str(row['nomination_date'])
    wait_data_export.append(row)
json.dump(wait_data_export, open('./joined_judges_wait.json', 'w'))

In [4]:
pwd

'/src/notebooks'

### Education

In [82]:
judges_in_year_query = """
    SELECT
        year_party.year,
        COUNT(appointment.start_year) AS count,
        ARRAY_AGG(appointment.nid) AS nid_serving_in_year
    FROM year_party
    LEFT JOIN appointment
        ON 
         year_party.party = appointment.party_of_appointing_president
         AND
        (
            appointment.start_year < year_party.year + 2
            AND
            (
                appointment.end_year >= year_party.year
                OR appointment.end_year IS NULL
            )
        )
    GROUP BY year_party.year
    ORDER BY year_party.year
"""

session.rollback()
judges_in_year = session.execute(judges_in_year_query)
jiy_data = [dict(row) for row in judges_in_year]

In [86]:
education_per_judge = session.execute("SELECT * FROM education")
education_data = [dict(row) for row in education_per_judge]

In [87]:
# Flatten so every judge is represented once but can have multiple educations
education_data_flat = defaultdict(list)
for row in education_data:
    education_data_flat[row['nid']].append(row)

In [89]:
jiy_data_with_ed_count = []
for row in jiy_data:
    ed_count_jd = Counter()
    ed_count_all = Counter()
    for nid in row['nid_serving_in_year']:
        for education in education_data_flat[nid]:            
            ed_count_all[education['school']] += 1
            if education['degree'] and education['degree'].lower() == 'j.d.':
                ed_count_jd[education['school']] += 1
    row['school_count_all'] = [{'id': k, 'count': v} for k, v in ed_count_all.items()]
    row['school_count_jd'] = [{'id': k, 'count': v} for k, v in ed_count_jd.items()]
    jiy_data_with_ed_count.append(row)
json.dump(jiy_data_with_ed_count, open('/src/js-ui/education_counts_by_year.json', 'w'))

### Unsuccessful Nominations

Taken from another dataset, so unique identifers like JID and NID are not present

Every row is represents a judicial nomination that was unsuccessful . It contains:

* Nominating President
* Congress of nomination year
* Judge Name
* Court Name
* Nomination Date
* Outcome



In [113]:
session.rollback()
unsuccesful_nominations = session.execute(
"""
SELECT *,
CASE WHEN nominee like '%(1)%' OR nominee like '%(2)%' THEN true ELSE false END AS later_appointed
FROM unsuccessful_nomination un JOIN congress c ON un.congress_start_year = c.start_year
""")
unsuccesful_nominations_export = []

for row in unsuccesful_nominations:
    nom = dict(row)
    nom['nomination_date'] = str(nom['nomination_date'])
    unsuccesful_nominations_export.append(nom)

json.dump(unsuccesful_nominations_export, open('./unsuccessful_nominations.json', 'w'))

In [103]:
dict(row)

{'president': 'George Washington',
 'congress': '1st',
 'congress_start_year': 1789,
 'congress_end_year': 1791,
 'nominee': 'Harrison, Robert H.',
 'court_name': 'Supreme Court',
 'nomination_date': datetime.date(1789, 9, 24),
 'recess_appointment': False,
 'outcome': 'confirmed September 26, 1789; nominee declined'}

In [98]:
unsuccesful_nominations

[]